In [ ]:
%load_ext autoreload
%autoreload 2

import os
from os.path import join as pjoin
import pandas as pd
import seaborn as sns
import numpy as np
from datetime import datetime
import pingouin as pg
import matplotlib.pyplot as plt
import seaborn as sns

from bmp_config import path_data, envcode2env
from bmp_behav_proc import *

fnf = pjoin(path_data,'df_all_multi_tsz__.pkl.zip')
print(fnf)
print( str(datetime.fromtimestamp(os.stat(fnf).st_mtime)))
df_all_multi_tsz = pd.read_pickle(fnf)
print(df_all_multi_tsz['trial_group_col_calc'].unique())
print(df_all_multi_tsz['retention_factor_s'].unique())
df_all_multi_tsz_orig = df_all_multi_tsz

In [ ]:
df = df_all_multi_tsz.query('trial_shift_size == 1 and trial_group_col_calc == "trialwe" '
                           ' and retention_factor_s == "0.924"').copy().sort_values(['subject','trials'])
df_orig = df
# define dfall here
df,dfall,ES_thr,envv,pert = addBehavCols2(df);


In [ ]:
assert df_orig.groupby('subject').size().min() == 768

In [ ]:
print( df_all_multi_tsz['trial_group_col_calc'].unique() )
checkErrBounds(df,['error','prev_error','error_deg'])
assert dfall.query('env == "stable"').trialwpertstage_wb.max() < 60

# Fig 2 CD

In [ ]:
from figure import renameTickLabels, palette_stabrand, subenv2color, env_order, subenv_order, palette_subenv
from figure.mystatann import plotSigAll, plotSig0All
from figure.imgfilemanip import stackSVGandShowJupy, svg2png
text_shift = (-60, 20)
annot_loc = (0, 1.03)
mcorr = 'holm'
thr = "mestd*0"
me = dfall.groupby(['thr','subject','env'], observed=True).\
    mean(numeric_only=1).reset_index()
me_env = me
#me.groupby(['env','thr']).size()
sns.set(font_scale=1.3)
fg = sns.catplot(data = me.query('thr == @thr'), 
                 kind='violin', y='err_sens', 
    x='env', hue='env', order = env_order, hue_order=env_order,
                palette = palette_stabrand)
#addTitleInfo(fg.axes.flatten()[0])
for ax in fg.axes.flatten():
    ax.axhline(y=0, c='r', ls=':'); #ax.set_ylim(-5,5)
    
ylast, ttrssig_env = plotSigAll(ax, 0.83, 0.05, ticklen=0.02,
       df=me, coln='err_sens', colpair = 'env', multi_comp_corr_method=mcorr)
    
ax.annotate('C', xy=annot_loc, xytext=text_shift, 
      fontsize=19, fontweight='bold', va='top', ha='left',
      xycoords='axes fraction', textcoords='offset points')
fg.set_ylabels('Error sensitivity')
ax.set_xlabel('Environment')
plt.title('Error sensititvity\nand environment stability')
ax.get_figure().subplots_adjust(bottom=0.3, top=0.9)

    
###################
# non-pooled stages and env comparison
me = dfall.groupby(['thr','subject','ps2_'], observed=True).\
    mean(numeric_only=1).reset_index()
me_ps = me

# if we want to add stuff ot 2C
#plot_err_sens_by_ps2(ax, me_ps, subenv2color, ps_2nice, ps2_col='ps2_', err_sens_col='err_sens')
fign = 'Fig2C'
plt.savefig(pjoin(path_fig, 'behav',fign + '.svg'))
plt.savefig(pjoin(path_fig, 'behav',fign + '.pdf'))
#plt.show()
plt.close()

##########################

sns.set(font_scale=1.3)
fg = sns.catplot(data = me.query('thr == @thr'), kind='violin', y='err_sens', 
    x='ps2_', order=subenv_order, hue_order= subenv_order,
                 palette = palette_subenv,
                hue = 'ps2_')
for ax in fg.axes.flatten():
    ax.axhline(y=0, c='r', ls=':'); #ax.set_ylim(-5,5)

ylast, ttrssig_ps = plotSigAll(ax, 2.05, 0.14, ticklen=0.05,
       df=me, coln='err_sens', colpair = 'ps2_', fontsize = 10, multi_comp_corr_method=mcorr)

ax.annotate('D', xy=annot_loc, xytext=text_shift, 
      fontsize=19, fontweight='bold', va='top', ha='left',
      xycoords='axes fraction', textcoords='offset points')

ax.set_xlabel('Experiment stage')
ps_2nice = dict( zip(['pre','pert','washout','rnd'], 
        ['No perturbation','Perturbation','Washout','Random']) )
renameTickLabels(ax, ps_2nice, rotation=30)
ax.get_figure().subplots_adjust(bottom=0.3, top=0.9)

fg.set_ylabels('Error sensitivity')
plt.title('Error sensititvity\nand experiment stages')

fign = 'Fig2D'
plt.savefig(pjoin(path_fig, 'behav',fign + '.svg'))
plt.savefig(pjoin(path_fig, 'behav',fign + '.pdf'))
plt.close()

fign = 'Fig2C'
svg_files_hor = [pjoin(path_fig, 'behav',fign + '.svg')]
fign = 'Fig2D'
svg_files_hor += [pjoin(path_fig, 'behav',fign + '.svg')]

fnfout_hor = pjoin(path_fig, 'behav',pjoin(path_fig, 'behav','Fig2CD' + '.svg'))
stackSVGandShowJupy(svg_files_hor, 'horizontal', fnfout_hor)
svg2png(fnfout_hor)

In [ ]:
# how would it look with more refined random?
dfall['ps5_'] = dfall['ps2_']
dfall.loc[ (dfall['ps2_'] == 'rnd') & (dfall['trialwb'] < 192. / 3.) ,'ps5_'] = 'subrnd_1'
dfall.loc[ (dfall['ps2_'] == 'rnd') & (dfall['trialwb'] >= 192. / 3.) & (dfall['trialwb'] < 192. * 2. / 3.) ,'ps5_'] = 'subrnd_2'
dfall.loc[ (dfall['ps2_'] == 'rnd') & (dfall['trialwb'] >= 192. * 2. / 3.) ,'ps5_'] = 'subrnd_3'

#[a for a in dfall.columns if a.find('ps') >= 0]
me = dfall.groupby(['thr','subject','ps5_'], observed=True).\
    mean(numeric_only=1).reset_index()
me_ps = me
#me.groupby(['ps_','thr']).size()
sns.set(font_scale=1.3)
fg = sns.catplot(data = me.query('thr == @thr'), kind='violin', y='err_sens', 
    x='ps5_', order=['pre','pert','washout','subrnd_1','subrnd_2','subrnd_3'],
                hue = 'ps5_')
for ax in fg.axes.flatten():
    ax.axhline(y=0, c='r', ls=':'); #ax.set_ylim(-5,5)

ylast, ttrssig_ps = plotSigAll(ax, 2.05, 0.14, ticklen=0.05,
       df=me, coln='err_sens', colpair = 'ps5_', fontsize = 10, multi_comp_corr_method=mcorr)

ttrs0,ttrs0_sig = plotSig0All(ax, 4.3, txt=None, df=me, 
          coln = 'err_sens', colpair = 'ps5_',
         multi_comp_corr_method=mcorr, graded_signif = True, show_ns = True)

ps_5nice = dict( zip(['pre','pert','washout','subrnd_1', 'subrnd_2','subrnd_3'], 
        ['No perturbation','Perturbation','Washout','Random 1', 'Random 2', 'Random 3']) )
renameTickLabels(ax, ps_5nice, rotation=45)

## Stats

In [ ]:
def format_pvalue(p):
    return f'{p:.2e}'

In [ ]:
# compare pairs: ANOVA + pairwise with correction
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.anova import AnovaRM
from statsmodels.stats.multitest import multipletests
from scipy.stats import ttest_rel
from itertools import combinations
mcorr = 'holm'
condition_labels = ps_2nice 

# Aggregate the data by taking the mean for each subject and condition
df_agg = dfall.groupby(['subject_ind', 'ps2_'], as_index=False)['err_sens'].mean()
# Perform Repeated Measures ANOVA on the aggregated data (it does not compute effect size so I use pingouin below)
rm_anova = AnovaRM(df_agg, 'err_sens', 'subject_ind', within=['ps2_']).fit()
print(rm_anova)
# f_value = rm_anova.anova_table['F Value'][0]
# df_effect = rm_anova.anova_table['Num DF'][0] # numerator
# df_error = rm_anova.anova_table['Den DF'][0]  # denumerator
# p_value = rm_anova.anova_table['Pr > F'][0]

aov = pg.rm_anova(dv='err_sens', within='ps2_', subject='subject_ind', data=dfall, detailed=True)
display(aov)
eta_squared_partial = aov.loc[0,'ng2']
print(f"Partial Eta-Squared: {eta_squared_partial:.3f}")

# Now perform pairwise t-tests for post-hoc comparisons
conditions = df_agg['ps2_'].unique()
pairwise_comparisons = list(combinations(conditions, 2))

# Store the results of pairwise t-tests
p_values = []
comparison_results = []

for cond1, cond2 in pairwise_comparisons:
    data1 = df_agg[df_agg['ps2_'] == cond1]['err_sens']
    data2 = df_agg[df_agg['ps2_'] == cond2]['err_sens']
    
    # Perform paired t-test (since the same subjects are in both groups)
    t_stat, p_val = ttest_rel(data1, data2)
    
    # Store results
    p_values.append(p_val)
    comparison_results.append((cond1, cond2, t_stat, p_val))

# Correct for multiple comparisons using Bonferroni correction
reject, pvals_corrected, _, _ = multipletests(p_values, method=mcorr)

# Prepare a human-readable report
significant_report = []
nonsignificant_report = []

for i, (cond1, cond2, t_stat, p_val) in enumerate(comparison_results):
    mean1 = df_agg[df_agg['ps2_'] == cond1]['err_sens'].mean()
    mean2 = df_agg[df_agg['ps2_'] == cond2]['err_sens'].mean()
    std1 = df_agg[df_agg['ps2_'] == cond1]['err_sens'].std()
    std2 = df_agg[df_agg['ps2_'] == cond2]['err_sens'].std()
    
    # Format p-values
    formatted_pval = format_pvalue(pvals_corrected[i])
    
    if reject[i]:
        if mean1 > mean2:
            significant_report.append(f"Error sensitivity in {condition_labels[cond1]} (mean={mean1:.2f} ± {std1:.2f}) is significantly higher "
                                      f"than in {condition_labels[cond2]} (mean={mean2:.2f} ± {std2:.2f}) t={t_stat:.2f} with p-value={formatted_pval}.")
        else:
            significant_report.append(f"Error sensitivity in {condition_labels[cond2]} (mean={mean2:.2f} ± {std2:.2f}) is significantly higher "
                                      f"than in {condition_labels[cond1]} (mean={mean1:.2f} ± {std1:.2f}) t={t_stat:.2f} with p-value={formatted_pval}.")
    else:
        nonsignificant_report.append(f"There is no significant difference in Error sensitivity between {condition_labels[cond1]} "
                                     f"(mean={mean1:.2f} ± {std1:.2f}) and {condition_labels[cond2]} "
                                     f"(mean={mean2:.2f} ± {std2:.2f}) t={t_stat:.2f} with p-value={formatted_pval}.")

# Combine significant and nonsignificant reports
final_report = "\n".join(significant_report + nonsignificant_report)

# Print or return the final report
print(final_report)

## compare ES values between themselves and 0

In [ ]:
# with env as condition
ttrs = []
for env in me_env.env.unique():
    r = compare0(me_env.query('env == @env'),'err_sens',cols_addstat=['err_sens'])
    r['env'] = env
    ttrs += [r]
ttrs_pos = pd.concat(ttrs).query('pval <= 0.05').set_index('env')
assert len(ttrs_pos) == 2
display(ttrs_pos)

s0 = 'Averages of ES within participant. \n'
for env in me_env.env.unique():
    row = ttrs_pos.loc[env]
    s = (f"{env} ES mean = {row['err_sens_mean']:.2f} (std = {row['err_sens_std']:.2f}),"
        f" ES > 0 p-value = {row['pval']:.2e}. \n" )
    s = s[0].upper() + s[1:]
    s0 += s

display(ttrssig_env)
row = ttrssig_env.iloc[0]

s = f"Stable > random T={row['T']:.2f}, p-value={row['pval']:.2e}"
print(s0)
print(s)

In [ ]:
# compare with zero
mcorr = 'holm'

me_ps = dfall.groupby(['thr','subject','ps2_'], observed=True).\
    mean(numeric_only=1).reset_index()

from bmp_behav_proc import multi_comp_corr
ttrs = []
for ps in me_ps.ps2_.unique():
    r = compare0(me_ps.query('ps2_ == @ps'),'err_sens',cols_addstat=['err_sens'])
    r['ps2_'] = ps
    ttrs += [r]
ttrs = multi_comp_corr(pd.concat(ttrs).reset_index(), method=mcorr )
ttrs_pos = ttrs.query('pval <= 0.05')
assert len(ttrs_pos) == 4
display(ttrs_pos)
ttrs_pos = ttrs_pos.set_index('ps2_')

s0 = 'Averages of ES within participant: \n'
for ps in me_ps.ps2_.unique():
    row = ttrs_pos.loc[ps]
    s = (f"{ps} ES mean = {row['err_sens_mean']:.2f} (std = {row['err_sens_std']:.2f}),"
        f" ES > 0 p-value = {row['pval']:.2e}. \n" )
    s = s[0].upper() + s[1:]
    s0 += s

#display(ttrssig_ps)
#row = ttrssig_ps.iloc[0]

ttrssig_ps,_ = comparePairs(me_ps, 'err_sens', 'ps2_', alt=['greater'], 
                            paired=True, updiag=False)
display(ttrssig_ps[ttrssig_ps.columns[-5:]])
s1 = ''
for i,row in ttrssig_ps.iterrows():
    ttstr = row['ttstr']
    s = ttstr.replace(row['val2'], ps_2nice[row['val2']] ).replace(row['val1'],
                                                                   ps_2nice[row['val1']] )    
    s += f" T={row['T']:.2f}, p-value={row['pval']:.2e}; \n"
    s1 += s
#s = f"Stable > random p-value={row['pval']:.2e}"
print(s0)
print(s1)

s = ''
for cond, row in ttrs_pos.iterrows():
    # Transform the index using ps2_nice dictionary
    condn = ps_2nice.get(cond, row['index'])  # Fallback to original if not found in dict
    
    # Print the desired output for each row
    s += f"during {condn} condition error sensitivity={row['err_sens_mean']:.2f} ± {row['err_sens_std']:.2f}, t={row['T']:.2f}, p={row['pval']:.2e}, "
print(s.capitalize()[:-2]+'.')

In [ ]:
# show that there is no difference between pert and rand 
psvals = ["pert","rnd"]
ttrssig,ttrssig_all = comparePairs(me_ps.query('ps2_.isin(@psvals)'),
                         'err_sens', 'ps2_', alt=['two-sided'], 
                        paired=True, updiag=True, pooled=0)
display(ttrssig_all)
assert len(ttrssig_all) == 1
row = ttrssig_all.iloc[0]
if row['pval'] > 0.05:    
    s = 'The difference between ES in '
    s += '{} and {} conditions is not significant ('.format( 
        ps_2nice[row['val2']], ps_2nice[row['val1'] ] )
    s += f" T={row['T']:.2f}, p-value={row['pval']:.2e})."
    #s1 += s
    print(s)
else:
    print('Actually there is significance')

# Fig 2 AB

In [ ]:
from figure.plots import Fig2_annotate_segments
tr = np.arange(len(pert))
c = 1 / np.pi  * 180
svg_files = []
qs = None
for lablet,varn,pertc,figsz,y_tl,y_title in [('A', 'error_pscadj', 1.,(12,10),1.15, 1.48), 
                   ('B', 'err_sens', 0.06,(12,4),1.,1.)]:

    varn_eff = varn
    if varn.startswith('org_feedback'):
        dfall['vtp'] = np.array(dfall[varn]) * c# - 180
        varn_eff = 'vtp'
    elif varn.startswith('error'):
        dfall['vtp'] = np.array(dfall[varn]) * c 
        varn_eff = 'vtp'

    plt.figure(figsize=figsz)
    fg = sns.relplot(data=dfall,
        kind='line', x="trials", y=varn_eff, 
        col='thr', estimator='mean', errorbar='sd', height = 4, aspect=2.5)

    if varn == 'error_pscadj':
        Fig2_annotate_segments(fg.ax, dfall, qs, category_col='ps2_', rotation_angle=60,
            ps_2nice = ps_2nice, text_size = 12, text_y_position = 35, x_shift=-8)
    else:
        Fig2_annotate_segments(fg.ax, dfall, qs, category_col='ps2_', 
            ps_2nice = ps_2nice, show_text = False)


    for ax in fg.axes.flatten():
        ax.plot(tr, pert * pertc, c='violet', ls= '--', lw=4)
        #ax.plot(tr, envv * 30 * pertc, c='black', lw=0, marker='.' ,alpha=0.2)
        #ax.set_title(f'{varn}  {truncs}')
        ax.set_ylabel(varn )
    if varn.startswith('error_'):
        fg.set_ylabels('Angular error [deg]')
        fg.set_titles('Error dynamics', y =y_title)
    else:
        fg.set_ylabels('Error sensitivity')
        fg.set_titles('Error sensitivity dynamics')
    fg.set_xlabels('Trial number')
 
    #addTitleInfo(fg.axes.flatten()[0])
    ax.annotate(lablet, xy=(0, y_title), xytext=(-60, 20), 
      fontsize=19, fontweight='bold', va='top', ha='left',
      xycoords='axes fraction', textcoords='offset points')

    fign = 'Fig2'+lablet
    fnfbase = pjoin(path_fig, 'behav',fign )
    fnfsvg = fnfbase + '.svg'
    plt.tight_layout(rect=[0,0,1,y_tl])
    #plt.tight_layout()
    plt.savefig(fnfsvg, bbox_inches='tight')
    plt.savefig(fnfbase + '.pdf')
    plt.close()

    svg_files += [fnfsvg]

from figure.imgfilemanip import *
#svg_files = [pjoin(path_fig, 'behav','Fig2'+lablet + '.svg') for lablet in ['A','B'] ]
print(svg_files)
fnfout = pjoin(path_fig, 'behav',pjoin(path_fig, 'behav','Fig2AB' + '.svg'))
stackSVGandShowJupy(svg_files, 'vertical', fnfout)

In [ ]:
display(svg_files)

In [ ]:
fnfout0 = pjoin(path_fig, 'behav',pjoin(path_fig, 'behav','Fig2AB' + '.svg'))
fnfout = pjoin(path_fig, 'behav', pjoin(path_fig, 'behav','Fig2ABCD' + '.svg'))
stackSVGandShowJupy([fnfout0, fnfout_hor], 'vertical', fnfout)

svg2png(fnfout, dpi=300)